Philippe JOly
This is to further tune hyperparams using a simple grid search (instead of Byaesian search)

In [53]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime as dt
import pickle

In [54]:
train = pd.read_csv('../../data/train_static.csv')
val = pd.read_csv('../../data/val_static.csv')
test = pd.read_csv('../../data/test_static.csv')

In [55]:

train.drop(columns=['Date/Time (UTC)','Year'], inplace=True)
val.drop(columns=['Date/Time (UTC)','Year'], inplace=True)
test.drop(columns=['Date/Time (UTC)','Year'], inplace=True)

X = train.drop(columns=["Average Power Output (MW)"]).values
y = train["Average Power Output (MW)"].values

X_val = val.drop(columns=["Average Power Output (MW)"]).values
y_val = val["Average Power Output (MW)"].values

X_test = test.drop(columns=["Average Power Output (MW)"]).values
y_test = test["Average Power Output (MW)"].values

In [56]:
y = y.reshape(len(y), 1)
y_val = y_val.reshape(len(y_val), 1)
y_test = y_test.reshape(len(y_test), 1)

x_scaler = StandardScaler().fit(X)
y_scaler = StandardScaler().fit(y)

X = x_scaler.transform(X)
y = y_scaler.transform(y)

X_val = x_scaler.transform(X_val)
y_val = y_scaler.transform(y_val)

X_test = x_scaler.transform(X_test)
y_test = y_scaler.transform(y_test)

y = y.reshape(y.shape[0])
y_val = y_val.reshape(y_val.shape[0])
y_test = y_test.reshape(y_test.shape[0])

# Hype-Parameter Tuning

In [57]:
svr = SVR()

In [72]:
opt_params=[{
    'C': np.logspace(0,3,6),
    'gamma': np.logspace(-3,1,6),
    'epsilon': [ 0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7],
    'kernel': ['rbf'],
    'max_iter': [1000]
}]

In [73]:
clf = GridSearchCV(estimator=svr, param_grid=opt_params, verbose=4, cv=3, n_jobs=-1)
clf.fit(X,y)

Fitting 3 folds for each of 252 candidates, totalling 756 fits


c:\Users\philj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


GridSearchCV(cv=3, estimator=SVR(), n_jobs=-1,
             param_grid=[{'C': array([   1.        ,    3.98107171,   15.84893192,   63.09573445,
        251.18864315, 1000.        ]),
                          'epsilon': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7],
                          'gamma': array([1.00000000e-03, 6.30957344e-03, 3.98107171e-02, 2.51188643e-01,
       1.58489319e+00, 1.00000000e+01]),
                          'kernel': ['rbf'], 'max_iter': [1000]}],
             verbose=4)

In [60]:
results = pd.DataFrame(clf.cv_results_).sort_values(by="rank_test_score", ascending=True)

In [61]:
clf.best_score_

0.8401716475850526

In [75]:
results.head(25)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_epsilon,param_gamma,param_kernel,param_max_iter,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
30,8.789857,0.946650,8.946734,0.758419,1.0,0.4,0.01,rbf,2000,"{'C': 1.0, 'epsilon': 0.4, 'gamma': 0.01, 'ker...",0.876683,0.858238,0.819256,0.851392,0.023939,1
35,5.836779,0.088296,6.268746,0.619768,1.0,0.5,0.01,rbf,2000,"{'C': 1.0, 'epsilon': 0.5, 'gamma': 0.01, 'ker...",0.875683,0.842188,0.830498,0.849456,0.019149,2
32,3.923333,0.198960,4.082647,0.512869,1.0,0.4,0.1,rbf,2000,"{'C': 1.0, 'epsilon': 0.4, 'gamma': 0.1, 'kern...",0.821430,0.923145,0.802937,0.849171,0.052850,3
31,5.835417,0.317157,6.597790,0.498358,1.0,0.4,0.031623,rbf,2000,"{'C': 1.0, 'epsilon': 0.4, 'gamma': 0.03162277...",0.829973,0.901166,0.810435,0.847191,0.038991,4
37,2.654322,0.345554,2.237667,0.358808,1.0,0.5,0.1,rbf,2000,"{'C': 1.0, 'epsilon': 0.5, 'gamma': 0.1, 'kern...",0.821457,0.907524,0.804479,0.844487,0.045110,5
36,4.031229,0.572286,3.177150,0.412871,1.0,0.5,0.031623,rbf,2000,"{'C': 1.0, 'epsilon': 0.5, 'gamma': 0.03162277...",0.845971,0.884037,0.798558,0.842855,0.034966,6
56,1.067689,0.089019,0.148517,0.011109,100.0,0.8,0.031623,rbf,2000,"{'C': 100.0, 'epsilon': 0.8, 'gamma': 0.031622...",0.862465,0.834603,0.801600,0.832889,0.024878,7
41,0.606356,0.141556,0.350404,0.046793,1.0,0.8,0.031623,rbf,2000,"{'C': 1.0, 'epsilon': 0.8, 'gamma': 0.03162277...",0.845192,0.810586,0.837254,0.831011,0.014802,8
55,1.193512,0.105939,0.203339,0.002061,100.0,0.8,0.01,rbf,2000,"{'C': 100.0, 'epsilon': 0.8, 'gamma': 0.01, 'k...",0.862481,0.833490,0.771680,0.822550,0.037868,9
72,1.006260,0.162402,0.196014,0.045196,10000.0,0.8,0.1,rbf,2000,"{'C': 10000.0, 'epsilon': 0.8, 'gamma': 0.1, '...",0.845738,0.832557,0.786027,0.821441,0.025613,10


# Further Tuning

In [76]:
opt_params=[{
    'C': np.logspace(0,3,6),
    'gamma': np.logspace(-4,-2,6),
    'epsilon': [  0.4, 0.5,0.6,0.7],
    'kernel': ['rbf'],
    'max_iter': [1000]
}]

In [77]:
clf = GridSearchCV(estimator=svr, param_grid=opt_params, verbose=4, cv=3, n_jobs=-1)
clf.fit(X,y)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


c:\Users\philj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


GridSearchCV(cv=3, estimator=SVR(), n_jobs=-1,
             param_grid=[{'C': array([   1.        ,    3.98107171,   15.84893192,   63.09573445,
        251.18864315, 1000.        ]),
                          'epsilon': [0.4, 0.5, 0.6, 0.7],
                          'gamma': array([0.0001    , 0.00025119, 0.00063096, 0.00158489, 0.00398107,
       0.01      ]),
                          'kernel': ['rbf'], 'max_iter': [1000]}],
             verbose=4)

In [78]:
results = pd.DataFrame(clf.cv_results_).sort_values(by="rank_test_score", ascending=True)

In [79]:
clf.best_score_

0.8453451497152004

In [80]:
results.head(25)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_epsilon,param_gamma,param_kernel,param_max_iter,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
87,2.440346,0.228517,2.185198,0.032644,63.095734,0.6,0.001585,rbf,1000,"{'C': 63.0957344480193, 'epsilon': 0.6, 'gamma...",0.887776,0.819151,0.829108,0.845345,0.030277,1
116,2.107552,0.400342,1.532001,0.024912,251.188643,0.7,0.000631,rbf,1000,"{'C': 251.18864315095797, 'epsilon': 0.7, 'gam...",0.865927,0.795255,0.868469,0.843217,0.033930,2
11,5.075634,0.253291,4.426424,0.139790,1.0,0.5,0.01,rbf,1000,"{'C': 1.0, 'epsilon': 0.5, 'gamma': 0.01, 'ker...",0.875136,0.828908,0.823122,0.842389,0.023276,3
64,2.253340,0.112748,2.202334,0.189956,15.848932,0.6,0.003981,rbf,1000,"{'C': 15.848931924611133, 'epsilon': 0.6, 'gam...",0.857472,0.835496,0.830940,0.841303,0.011584,4
17,3.073004,0.168111,3.298989,0.296895,1.0,0.6,0.01,rbf,1000,"{'C': 1.0, 'epsilon': 0.6, 'gamma': 0.01, 'ker...",0.869105,0.819696,0.828678,0.839160,0.021490,5
47,1.635842,0.151795,1.235175,0.105232,3.981072,0.7,0.01,rbf,1000,"{'C': 3.9810717055349722, 'epsilon': 0.7, 'gam...",0.861437,0.825083,0.830699,0.839073,0.015979,6
141,1.021669,0.083228,0.527330,0.061648,1000.0,0.7,0.001585,rbf,1000,"{'C': 1000.0, 'epsilon': 0.7, 'gamma': 0.00158...",0.867982,0.815832,0.833391,0.839068,0.021665,7
40,3.176387,0.355407,3.732414,0.414530,3.981072,0.6,0.003981,rbf,1000,"{'C': 3.9810717055349722, 'epsilon': 0.6, 'gam...",0.873323,0.806623,0.835215,0.838387,0.027323,8
70,1.691999,0.133823,1.324999,0.092537,15.848932,0.7,0.003981,rbf,1000,"{'C': 15.848931924611133, 'epsilon': 0.7, 'gam...",0.870555,0.814939,0.825398,0.836964,0.024133,9
41,2.302030,0.190590,2.260388,0.252839,3.981072,0.6,0.01,rbf,1000,"{'C': 3.9810717055349722, 'epsilon': 0.6, 'gam...",0.868428,0.844941,0.793326,0.835565,0.031369,10


In [83]:
model_ls = []
err = []
c = results['param_C']
e = results['param_epsilon']
g = results['param_gamma']
for i in range(10):
    err.append([])
    model = SVR(C=c.iloc[i], epsilon=e.iloc[i], gamma=g.iloc[i], kernel='rbf',max_iter=3000)
    model.fit(X, y)
    y_pred = model.predict(X_val)
    err[i].append(mean_squared_error(y_val, y_pred))
    err[i].append(r2_score(y_val, y_pred))
    err[i].append(mean_absolute_error(y_val, y_pred))
    model_ls.append(model)

c:\Users\philj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\philj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\philj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\philj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\U

In [82]:
err

[[0.1353825671774423, 0.8618228285236875, 0.29712166344103985],
 [0.13478701281362157, 0.8624306764849783, 0.29982840756915813],
 [0.11514739536167924, 0.8824757003382295, 0.27761536579377616],
 [0.10394996524690853, 0.8939042708943986, 0.26438072447173006],
 [0.10986322739269463, 0.8878689455601423, 0.272257192589893],
 [0.10799907493297235, 0.889771578369191, 0.2714100248962098],
 [0.15975192491262283, 0.8369504317835059, 0.32391068731912476],
 [0.11885241176222654, 0.8786942039670611, 0.28145063935522513],
 [0.12046262085008638, 0.8770507564990107, 0.286492064336748],
 [0.09450680490508673, 0.9035423595570254, 0.25185778248910595]]

In [85]:
err2 = []
model = SVR(C=2.782559402207126, epsilon=0.5, gamma=0.04641588833612782, kernel='rbf')
model.fit(X, y)
y_pred = model.predict(X_val)
err2.append(mean_squared_error(y_val, y_pred))
err2.append(r2_score(y_val, y_pred))
err2.append(mean_absolute_error(y_val, y_pred))
model_ls.append(model)

In [86]:
err2

[0.06394468628779569, 0.9347353498578067, 0.206752980239755]